In [1]:
import pandas as pd
import numpy as np
import re
import copy
import random

In [95]:
#open the file
cable_file = 'Diplomacy_W129.xlsx'
#cable_file = 'Missing.xlsx'
df = pd.read_excel(cable_file).T

In [96]:
class Cleaner:
    def __init__(self,cable_dataframe):
        self.df = cable_dataframe
        self.fd = self.df.T
        self.separate_rows()
        self.clean_all()
        self.cleaned = self.recompile()
        
    def separate_rows(self):
        self.turns = self.fd['turn']
        self.authors = self.fd['author civ']
        self.subjects = self.fd['subject civ']
        self.cables = self.fd['cable']
        try:
            self.categories = self.fd['Category']
            self.flags = self.fd['Flag']
            self.indx = self.df[0:1]
        except:
            pass
    
    def clean_all(self):
        self.clean_turn()
        self.clean_author()
        self.clean_subject()
        self.clean_cables()

    def clean_turn(self):
        pass
    def clean_author(self):
        #for i in range(len(self.authors)):
        for i in self.authors.keys():
            self.authors[i] = self.authors[i].replace('the','The')
    def clean_subject(self):
        #for i in range(len(self.subjects)):
        for i in self.subjects.keys():
            self.subjects[i] = self.subjects[i].replace('the','The')
    def clean_cables(self):
        bad_char = [',','!','.','"','\\',"'s'","'"]
        break_char = ['\r\n','\n','\r']
        #for i in range(len(self.cables)):
        for i in self.cables.keys():
            self.cables[i] = self.cables[i].replace('. ','\n')
            for j in bad_char:
                self.cables[i] = self.cables[i].replace(j,'')
            for j in break_char:
                self.cables[i] = self.cables[i].replace(j,'. ')
            self.cables[i] = self.cables[i].split('. ')
            

    def recompile(self):
        final = pd.DataFrame(self.turns).T
        final = final.append(self.authors)
        final = final.append(self.subjects)
        final = final.append(self.cables)
        try:
            final = final.append(self.categories)
            final = final.append(self.flags)
        except:
            pass
        return(final)

cdf = Cleaner(copy.copy(df))

In [70]:
cdf.cleaned

,40,242,338,367,419,522,546,853,913,1193,...,7326,7329,7416,7571,7583,7598,7663,7713,7855,7972
turn,11,28,36,38,42,49,50,67,72,87,...,444,444,448,462,462,463,466,470,481,491
author civ,Songhai,Morocco,Korea,Morocco,Germany,Morocco,Morocco,The Iroquois,Germany,The Iroquois,...,Germany,Germany,Korea,Germany,Korea,Germany,The Incas,Korea,Morocco,Songhai
subject civ,Morocco,The Iroquois,Germany,Germany,Songhai,The Incas,Korea,Songhai,Morocco,Germany,...,Korea,Korea,The Incas,Songhai,Morocco,Morocco,Germany,Songhai,Songhai,The Incas
cable,[The foreign office desires any intelligence p...,[The foreign office asks for any information o...,[The foreign office requires any knowledge on ...,[The foreign office requires any reporting in ...,[The foreign office asks for any information o...,[The foreign office requires any material avai...,[The foreign office requires any advice you ha...,[The foreign office requires any data in your ...,[The foreign office requires any advice obtain...,[The foreign office requires any data in your ...,...,[Urgent: we are now in an established state of...,[The foreign office desires any intelligence o...,[The foreign office asks for any news obtainab...,[The foreign office requires any information i...,[Welcome news: we have concluded violence with...,[The foreign office requires any data obtainab...,[Happy news: we have ceased violence with the ...,[ The foreign office desires any material obta...,[Insofar as we can assume the Songhai believes...,[The foreign office asks for any knowledge on ...
Category,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Flag,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,...,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing


In [142]:
class Dictionaries:
    def __init__(self,clean_object):
        self.cdf = clean_object
        self.define_signifiers()
        self.define_placenames()
        self.define_attributes()
        
        
        self.categorize_cables()
        
    def define_signifiers(self):
        self.signifiers = pd.DataFrame(columns = ['has_some','has_none']).T
        self.signifiers['Danger'] = [['risk','hazard','threat','problem','obstacle','danger'],['updates']]
        self.signifiers['Conquerability'] = [['conquering','for attack','dominating','taking over','target'],[]]
        self.signifiers['Economy'] = [['econom'],['any','foreign']]
        self.signifiers['Embassy'] = [['embassy','seat of government'],[]]
        self.signifiers['Greeting'] = [['wishes','reetin','honor','finds','I write','updates','alwyas'],['foreign']]
        self.signifiers['NA'] = [['no news'],[]]
        self.signifiers['Confidence'] = [['his assessment','in this','hypothesis','convinced','confid',
                                          'certain','very sure','sure source'],[]]
        self.signifiers['Info Request'] = [['would','any forecasting','Please','must have','wants any',
                                            'requests','needs any','asks for','desires any','requires any'],[]]
        self.signifiers['Military Strength'] = [['military'],['foreign','dominance ','military preeminence']]
        self.signifiers['War'] = [['war','conflict','the fighting','peace','news','Urgent','armistice'],
                                  ['foreign office','ward','preeminence','dominance','average','strong','poor',
                                   'weak','powerful','immense','pathetic']]
        self.signifiers['Trading'] = [['haggle','borders','trade','unit','barter','a deal','bargain','to deal'],
                                      ['embassy','seat of government','end the fighting']]
        self.signifiers['Prediction'] = [['preeminence','dominance'],[]]

                                                
        
        self.signifiers['Diplomatic Stance'] = [['Our relations with','protective','proceedings','policy','transactions','matters','guarded','hostile',
                                                 'aggressive','deceptive','afraid','friendly stance','neutral stance','friendly posture',
                                                 'neutral posture','likely neutral','seems neutral','looks neutral',
                                                 'friendly manner','neutral manner','friendly attitude','neutral attitude',
                                                 'feels neutral','feels friendly','a friend','A friend','an ally',
                                                 'now are neutral','are of late neutral','currently are neutral',
                                                 'these days are neutral','of late are neutral','at present are friendly',
                                                 'presently are neutral','at present neutral','are presently neutral',
                                                 'are currently neutral','are these days neutral','are now neutral',
                                                 'now are friendly','are of late friendly','currently are friendly',
                                                 'these days are friendly','of late are friendly','presently are friendly',
                                                 'at present friendly','are presently friendly','at present are neutral',
                                                 'are currently friendly','are these days friendly','are now friendly',
                                                 'likely friendly','seems friendly','looks friendly','appears neutral','appears friendly',
                                                 'protective position','afraid position','hostile position','friendly position',
                                                 'deceptive position','neutral position','guarded position',
                                                 'afraid disposition','hostile disposition','friendly disposition',
                                                 'deceptive disposition','neutral disposition','guarded disposition',
                                                 'protective disposition'],[]]
        self.signifiers['Diplomatic Opinion'] = [['civilization','party','nation','country','open posture','popular posture','outward posture',
                                                  'reputed posture','professed posture','announced posture',''
                                                    'dealings','competitor','enemy','archnemesis','disposition','position'],
                                                 ['protective','afraid','hostile','friendly position','friendly disposition',
                                                  'guarded','deceptive','neutral position','neutral disposition','likely neutral','seems neutral',
                                                  'looks neutral','likely friendly','seems friendly','looks friendly',
                                                 'appears neutral','appears friendly','feels friendly','feels neutral',
                                                  'appears friendly','appears neutral','neutral position','friendly position']]
 
    def define_placenames(self):
        self.countries = ['The Songhais','the Songhais','Songhai',
                         'The Germans','the Germans','Germany','German',
                         'The Koreans','the Koreans','Korean','Korea',
                         'The Moroccans','the Moroccans','Moroccan','Morocco',
                         'The Incas','the Incas','Incan','Inca',
                         'The Iroquois','the Iroquois','Iroquois']
        self.city_states = ['Bogota', 'Budapest', 'Florence', 'Ife', 'Kathmandu', 'Kyzyl',
                            'Mombasa', 'Panama City', 'Singapore', 'Yerevan']
    
    def define_attributes(self):
        self.data = self.cdf.cleaned
        self.turns = self.cdf.turns
        self.authors = self.cdf.authors
        self.subjects = self.cdf.subjects
        self.cables = self.cdf.cables
    
    def categorize_cables(self):
        data = copy.copy(self.data)
        cols = list(data.index)
        if len(cols) < 5:
            cols.extend(['Category','Flag'])
        self.cat_cab = pd.DataFrame(columns = cols).T
        for i in data:
            if len(data[i]['cable'])>1:
                for j in data[i]['cable'][2:len(data[i]['cable'])-3]:
                    if len(j)>2:
                        cat,flag = self.get_cat(j)
                        self.cat_cab[len(self.cat_cab.T)] = [data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag]
                        print([data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag])
            else:
                j = data[i]['cable'][0]
                cat,flag = self.get_cat(j)
                self.cat_cab[i] = [data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag]
                print([data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag])
    
    def get_cat(self,cable):
        cab = cable
        cat = []
        for i in self.signifiers:
            for j in self.signifiers[i][0]:
                if j in cab:
                    cat.append(i)
            acab = np.array(cat)
            cat = list(np.unique(acab))
            for j in self.signifiers[i][1]:
                if j in cab:
                    if i in cat:
                        cat.remove(i)
        if len(cat) == 0:
            flag = 'Missing'
            cat = ''
        elif len(cat) > 1:
            flag = 'Multiple'
            cat = cat[-1]
        else:
            flag = 'None'
            cat = cat[0]
        return(cat,flag)
    
    
    def generalize_cables(self):      
        pass
        
dic = Dictionaries(copy.copy(cdf))


[2, 'Germany', 'Songhai', 'The Songhais military potency has hit an average degree according to our intelligence officers', 'Military Strength', 'None']
[2, 'Germany', 'Songhai', 'Our reputed policy with respect to the Songhai feels friendly', 'Diplomatic Stance', 'None']
[2, 'Songhai', 'Ife', 'Ifes military potential has reached a poor threshold according to our diplomatic cables', 'Military Strength', 'None']
[3, 'Songhai', 'Germany', 'We estimate the Germans to be an average target for attack', 'Conquerability', 'None']
[3, 'Songhai', 'Germany', 'Our relations with the Germans at present are friendly', 'Diplomatic Stance', 'None']
[3, 'Korea', 'Mombasa', 'Our public stance toward Mombasa appears neutral', 'Diplomatic Stance', 'None']
[5, 'Morocco', 'The Incas', 'A neutral civilization is how our sources portrayed their rapport with the Incas', 'Diplomatic Opinion', 'None']
[6, 'The Incas', 'Morocco', 'Our relations with the Moroccans presently are friendly', 'Diplomatic Stance', 'No

[19, 'Korea', 'Songhai', 'We figure the Songhais military to be average', 'Military Strength', 'None']
[19, 'Korea', 'The Iroquois', 'Please inform on the Iroquois military strategies as the foreign office is most alarmed about the Iroquois actions', 'Info Request', 'None']
[19, 'Korea', 'The Iroquois', 'Recent events have led us to imagine the Iroquois as a poor military force', 'Military Strength', 'None']
[19, 'Korea', 'The Iroquois', 'Insofar as we can tell the Iroquois feels we are a neutral country', 'Diplomatic Opinion', 'None']
[19, 'Korea', 'The Iroquois', 'We perceive the Iroquois to be an average target for attack', 'Conquerability', 'None']
[19, 'Korea', 'The Iroquois', 'The Songhais military strength has hit a strong status according to the Germans agents', 'Military Strength', 'None']
[19, 'Germany', 'The Incas', 'We desired a deal to authorize a foreign service post in The Incass top city and thus suggested an accord to authorize an embassy in Germanys seat of government

[27, 'Germany', 'The Incas', 'We consider the Incas to be an average target for attack', 'Conquerability', 'None']
[27, 'Germany', 'The Incas', 'The problem that the Incas constitute is described as minor by our diplomatic cables', 'Danger', 'None']
[27, 'Germany', 'The Incas', 'Please expand on the Incas economic ambitions as the foreign office is quite distressed about the Incas conduct', 'Info Request', 'None']
[27, 'Germany', 'The Incas', 'The Koreans military power has approached an average level according to the Moroccans sources', 'Military Strength', 'None']
[27, 'Germany', 'The Iroquois', 'The hazard that the Iroquois show is represented as moderate by our intelligence officers', 'Danger', 'None']
[27, 'Germany', 'The Iroquois', 'The Koreans are exhibiting a friendly manner toward the Iroquois', 'Diplomatic Stance', 'None']
[27, 'Germany', 'The Iroquois', 'A friendly stance with the Iroquois is what we avow', 'Diplomatic Stance', 'None']
[27, 'Korea', 'Germany', 'We consider t

[31, 'The Iroquois', 'Germany', 'We perceive the hope of taking over the Germans to be average', 'Conquerability', 'None']
[31, 'Songhai', 'Kathmandu', 'Recent incidents have led us to esteem Kathmandu as an average military actor', 'Military Strength', 'None']
[32, 'Morocco', 'Korea', 'We perceive the Koreans to be a neutral party', 'Diplomatic Opinion', 'None']
[32, 'Morocco', 'Korea', 'We regard the Koreans to be a favorable target for attack', 'Conquerability', 'None']
[32, 'Morocco', 'Korea', 'The Koreans economic potential has attained a strong level according to the Songhais sources', 'Economy', 'None']
[32, 'Germany', 'Korea', 'Whatever the Iroquois popular manner may be the Moroccans see the Iroquois as a neutral country', 'Diplomatic Opinion', 'None']
[32, 'Germany', 'Korea', 'We believe the Koreans to be an average target for attack', 'Conquerability', 'None']
[32, 'Germany', 'Korea', 'We believe that the Koreans have become a minor danger to our activities', 'Danger', 'None

[37, 'Germany', 'Morocco', 'A friendly attitude with the Moroccans is what we maintain', 'Diplomatic Stance', 'None']
[37, 'Korea', 'The Iroquois', 'We are regularly reminded by the Iroquois behavior why we call the Iroquois a neutral party', 'Diplomatic Opinion', 'None']
[37, 'The Iroquois', 'Songhai', 'We disclosed that proceedings between us and the Songhai are now friendly', 'Diplomatic Stance', 'None']
[37, 'The Iroquois', 'Songhai', 'We figure the Songhais military to be average', 'Military Strength', 'None']
[37, 'The Iroquois', 'Songhai', 'Recent news have led the Moroccans to consider the Songhai as a strong military contender', 'Military Strength', 'None']
[37, 'The Iroquois', 'Budapest', ' We are showing a friendly manner toward Budapest', 'Diplomatic Stance', 'None']
[38, 'The Incas', 'Songhai', 'Recent happenings have led the Moroccans to imagine the Songhai as a strong military contender', 'Military Strength', 'None']
[38, 'The Incas', 'Songhai', 'We figure the Songhais e

...

[472, 'Germany', 'Songhai', 'A friendly attitude with the Moroccans is what the Incas profess', 'Diplomatic Stance', 'None']
[472, 'Germany', 'Songhai', 'We imagine the prospect of taking over the Songhai to be favorable', 'Conquerability', 'None']
[472, 'Germany', 'Songhai', 'This correspondence is to share with you that we are at war with the Songhai as of today', 'War', 'None']
[472, 'Germany', 'Songhai', 'Our diplomatic cables esteem the Songhai as a favorable target', 'Conquerability', 'None']
[472, 'Germany', 'Songhai', 'An archnemesis is how our diplomatic cables characterized their association with the Songhai', 'Diplomatic Opinion', 'None']
[472, 'Morocco', 'Panama City', 'We estimate Panama Citys military to be average', 'Military Strength', 'None']
[472, 'Morocco', 'Panama City', 'Our relations with Panama City these days are neutral', 'Diplomatic Stance', 'None']
[472, 'Morocco', 'Bogota', 'Our relations with Bogota at present are neutral', 'Diplomatic Stance', 'None']
[472

[478, 'Morocco', 'Kyzyl', 'Our advertised diplomacy vis-à-vis Kyzyl appears protective', 'Diplomatic Stance', 'None']
[479, 'Songhai', 'Morocco', 'The Koreans published that proceedings between the Koreans and the Moroccans are of late guarded', 'Diplomatic Stance', 'None']
[479, 'Songhai', 'Morocco', 'The foreign office needs any reporting possible concerning the Moroccans military potency', 'Info Request', 'None']
[479, 'Songhai', 'Morocco', 'Our outward policy in regard to the Moroccans looks friendly', 'Diplomatic Stance', 'None']
[479, 'Morocco', 'The Incas', 'The Germans economic force has hit a poor position according to the Koreans intelligence officers', 'Economy', 'Multiple']
[479, 'Morocco', 'The Incas', 'We are periodically reminded by the Incas behavior why we call the Incas a favorable civilization', 'Diplomatic Opinion', 'None']
[479, 'Morocco', 'The Incas', 'The foreign office must have any knowledge in your possession detailing the Incas military power', 'Info Request'

[484, 'Korea', 'Songhai', 'Please report on the Songhais military pursuits as the foreign office is acutely alarmed about the Songhais behavior', 'Info Request', 'None']
[484, 'Korea', 'Songhai', 'We are regularly reminded by the Songhais behavior why we call the Songhai an archnemesis', 'Diplomatic Opinion', 'None']
[484, 'The Incas', 'Panama City', 'We are displaying a neutral position toward Panama City', 'Diplomatic Stance', 'None']
[484, 'The Incas', 'Panama City', 'We estimate Panama City to be an average target for attack', 'Conquerability', 'None']
[484, 'Morocco', 'Ife', 'Recent actions have led us to consider Ife as a weak military agent', 'Military Strength', 'None']
[484, 'Morocco', 'Ife', 'We believe Ife to be a soft target for attack', 'Conquerability', 'None']
[484, 'Morocco', 'Bogota', 'A neutral attitude with Bogota is what we avow', 'Diplomatic Stance', 'None']
[485, 'The Incas', 'Korea', 'The Germans figure the Koreans military to be strong', 'Military Strength', 'No

[491, 'Korea', 'Songhai', 'Our relations with the Songhai presently are guarded', 'Diplomatic Stance', 'None']
[491, 'Korea', 'Germany', 'Please send word on the Germans military strategies as the foreign office is very troubled about the Germans activity', 'Info Request', 'None']
[491, 'Korea', 'Germany', 'Seeing as we can discern the Germans believes we are an archnemesis', 'Diplomatic Opinion', 'None']
[491, 'Morocco', 'Singapore', 'Our public policy with regard to Singapore appears protective', 'Diplomatic Stance', 'None']
[492, 'Songhai', 'Germany', 'Please report on the Germans military pursuits as the foreign office is most alarmed about the Germans activity', 'Info Request', 'None']
[492, 'Songhai', 'Germany', 'Our politicians presently see the Germans as an archnemesis', 'Diplomatic Opinion', 'None']
[492, 'Songhai', 'Germany', 'Our outward feeling vis-à-vis the Germans is likely deceptive', 'Diplomatic Stance', 'None']
[492, 'Songhai', 'Germany', 'The Koreans are taking a neu

[499, 'Morocco', 'Germany', 'The problem that the Germans constitute is labeled as minor by our spies', 'Danger', 'None']
[499, 'Germany', 'The Incas', 'Our representatives presently regard the Incas as an archnemesis', 'Diplomatic Opinion', 'None']
[499, 'Germany', 'The Incas', 'We consider the Incas economy to be poor', 'Economy', 'None']
[499, 'Germany', 'The Incas', 'The Incas military power has reached a weak degree according to the Moroccans spies', 'Military Strength', 'None']
[499, 'Germany', 'The Incas', 'We figure the Incas to be a bad target for attack', 'Conquerability', 'None']
[499, 'Korea', 'The Incas', 'Please report on the Incas economic plans as the foreign office is quite worried about the Incas activity', 'Info Request', 'None']
[499, 'Korea', 'The Incas', 'The Songhai are continually reminded by the Incas behavior why the Songhai call the Incas a competitor', 'Diplomatic Opinion', 'None']
[499, 'Morocco', 'Ife', 'Our relations with Ife of late are neutral', 'Diplom

In [154]:
a = dic.cat_cab.T

print(a.Category.unique())
cats = a.Category.unique()

for i in cats:
    b = a[a.Category == i]
    b.to_excel('categoried/'+i+'.xlsx')

['Military Strength' 'Diplomatic Stance' 'Conquerability'
 'Diplomatic Opinion' 'Economy' 'Prediction' 'Confidence' 'Danger'
 'Info Request' 'Embassy' 'Trading' 'War']


In [170]:
a[a.Category == 'War'][a['author civ'] == 'Morocco']

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,turn,author civ,subject civ,cable,Category,Flag
1967,129,Morocco,The Incas,This communique is to disclose to you that we ...,War,None
2065,136,Morocco,The Iroquois,We wished for 2 units of horse for the next 3...,War,Multiple
3173,194,Morocco,Korea,We write to state the sad news that we are at ...,War,None
4151,249,Morocco,Germany,This communique is to inform you that we are a...,War,None
4275,255,Morocco,The Iroquois,We write to reveal the dismal news that we are...,War,None
4568,270,Morocco,Germany,Cheerful news: we have ended war with the Germ...,War,None
4678,276,Morocco,The Iroquois,Joyful news: we have ceased conflict with the ...,War,None
4725,278,Morocco,Germany,This letter is to relate to you that we are at...,War,None
4729,278,Morocco,Korea,This letter is to inform you that we are at wa...,War,None
5136,298,Morocco,Korea,Happy news: we have ceased war with the Korean...,War,None


In [ ]:


# turn,sender,receiver,Civ 1,Civ 2,stat,value,original cable,


